# Requirements:
- Define a new type Tablet that takes a single attribute called model at instantiation
- The class should only support 3 types of models: lite, pro and max
- Each instance will also have base_storage and memory attributes, which will be set automatically depending on the
specified model; base_storage doubles at each model increment (i.e. 32 of lite, 128 for max), whereas memory increases
by 1 (i.e. 1 for lite, 3 for max)
- In addition, the Tablet class should enable users to expand the storage through 2 separate interfaces:
    - an add_storage() method, e.g. t1.add_storage(32) should add 32GB to the base_storage of the instance
    - direct attribute setter, e.g. t1.storage = 256, should ensure that the overall memory of the device is 256 by
    dynamically handling the split between base_storage and added_storage depending on the model
- All models should not exceed 1024GB in combined storage
- The Tablet type should also reflect the memory and base_storage of the device as a read-only attributes
- Tablet instances should have a representation that would make it easy to recreate the instance

In [345]:
from collections import namedtuple
BaseSpec = namedtuple("BaseSpec", ("base_storage", "memory"))

class Tablet:
    MODELS = { 
        'lite': BaseSpec(32,1),
        'pro': BaseSpec(64,2) , 
        'max': BaseSpec(128,3), 
    }

    def __init__(self, model):
        self.model = model
        self.added_storage = 0

        self._base_storage = self.MODELS[self.model].base_storage
        self._memory = self.MODELS[self.model].memory

    @property
    def base_storage(self):
        return self._base_storage

    @property
    def memory(self):
        return self._memory
    
    @property
    def model(self):
        return self._model

    @model.setter
    def model(self, value):
        if value.lower() not in self.MODELS.keys():
            raise ValueError('Unrecognized model')
        self._model = value.lower()

    def __repr__(self):
        return f"{type(self).__name__}(model={self.model!r}, base_storage={self.base_storage}, " \
               f"added_storage={self.added_storage}, memory={self.memory})" 

    def add_storage(self, value):
        self._check_memory_value(value)
        self._check_overall_memory(value)
        self.added_storage += value

    @property
    def storage(self):
        return sum((self.base_storage, self.added_storage))

    @storage.setter
    def storage(self, value):
        self._check_memory_value(value)
        self._check_overall_memory(value)
        if value < self.base_storage:
            self.added_storage = 0
        else:
            self.added_storage = value-self.base_storage

    def _check_memory_value(self, value):
        if not isinstance(value, int) or value<0:
            raise ValueError("Can't add that storage!")
        
    def _check_overall_memory(self, value):
        if sum((self.base_storage, self.added_storage, value)) > 1024:
            raise ValueError("Memory cannot exceed 1024")



In [347]:
t1 = Tablet("Lite")
t1

Tablet(model='lite', base_storage=32, added_storage=0, memory=1)

In [348]:
BaseSpec = namedtuple("BaseSpec", ("base_storage", "memory"))

In [355]:
try:
    Tablet("Batman")
except ValueError as err:
    print(f"ValueError: {err}")

ValueError: Unrecognized model


In [357]:
t1.add_storage(16)
t1

Tablet(model='lite', base_storage=32, added_storage=16, memory=1)

In [359]:
t1.storage

48

In [363]:
t1.storage = 256
t1

Tablet(model='lite', base_storage=32, added_storage=224, memory=1)

In [369]:
t2 = Tablet('pro')
t2.storage = 256
t2

Tablet(model='pro', base_storage=64, added_storage=192, memory=2)

In [373]:
try:
    t1.add_storage(2048)
except ValueError as err:
    print(f"ValueError: {err}")

ValueError: Memory cannot exceed 1024


In [378]:
try:
    t1.memory = 2
except AttributeError as err:
    print(f"AttributeError: {err}")

AttributeError: can't set attribute


In [382]:
try:
    t1.base_storage = 64
except AttributeError as err:
    print(f"AttributeError: {err}")

AttributeError: can't set attribute
